### 가게별 UserID, ItemID, Rating, Timestamp 뽑기
- user_review_id = {}
- review_json = {}
- image_json = {}
- item_meta = {}

In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
from selenium.webdriver.common.keys import Keys
import pandas as pd
import math
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

In [2]:
data = pd.read_csv("C:/Users/zhqhd/Desktop/BOAZ 공부 자료/ADV/data/Final Res.csv")
data = data[["상호명","상권업종중분류명","상권업종소분류명","도로명주소","역명","거리"]]
data["역"] = data["역명"].apply(lambda x : x.split(" ")[1])
data["역"] = data["역"].apply(lambda x : x.split("(")[0])
data["검색어"] = data.apply(lambda x : x["역"] + "역 " + str(x["상호명"]),axis=1)
data

,상호명,상권업종중분류명,상권업종소분류명,도로명주소,역명,거리,역,검색어
0,도전최강달인왕만두,한식,한식/백반/한정식,서울특별시 강동구 고덕로 333,5호선 상일동,373.362877,상일동,상일동역 도전최강달인왕만두
1,이때,한식,한식/백반/한정식,서울특별시 마포구 동교로38안길 7,경의중앙 홍대입구,360.468770,홍대입구,홍대입구역 이때
2,L.A.D,한식,한식/백반/한정식,서울특별시 마포구 잔다리로3안길 23,2호선 홍대입구,583.594176,홍대입구,홍대입구역 L.A.D
3,제이씨에스푸드,한식,한식/백반/한정식,서울특별시 관악구 신림로14길 3,2호선 봉천,860.540255,봉천,봉천역 제이씨에스푸드
4,비지트,한식,한식/백반/한정식,서울특별시 서초구 동광로18길 82,2호선 방배,810.975643,방배,방배역 비지트
...,...,...,...,...,...,...,...,...
135805,얼음과자,패스트푸드,아이스크림판매,서울특별시 강서구 금낭화로 135,5호선 방화,27.430119,방화,방화역 얼음과자
135806,오이소서울,패스트푸드,아이스크림판매,서울특별시 강동구 고덕로80길 123,5호선 상일동,983.009054,상일동,상일동역 오이소서울
135807,얼음왕국명일14,패스트푸드,아이스크림판매,서울특별시 강동구 상암로 214,5호선 굽은다리(강동구민회관앞),347.273163,굽은다리,굽은다리역 얼음왕국명일14
135808,얼음과자,패스트푸드,아이스크림판매,서울특별시 강동구 고덕로83길 6,5호선 상일동,1380.011573,상일동,상일동역 얼음과자


In [3]:
items = data["검색어"].head(20)
items

0     상일동역 도전최강달인왕만두
1           홍대입구역 이때
2        홍대입구역 L.A.D
3        봉천역 제이씨에스푸드
4            방배역 비지트
5        총신대입구역 쓰리에프
6             종각역 채움
7         선유도역 호구의주방
8             서초역 다향
9        압구정역 고향생막걸리
10      남부터미널역 든든한우家
11        신목동역 이삭토스트
12        아차산역 옥돌영양탕
13     몽촌토성역 양푼이김치찌개
14         애오개역 은진이네
15         오목교역 흥부보쌈
16         문정역 고수의족발
17     선정릉역 호랑이가만든족발
18      왕십리역 왕십리제일곱창
19           혜화역 커피빈
Name: 검색어, dtype: object

In [4]:
driver = webdriver.Chrome("chromedriver")
driver.maximize_window() # 창 크기 최대화

rating_df = pd.DataFrame(columns = ["UserID","ItemID","Rating","Timestamp"])
user_review_id = {}
review_json = {}
image_json = {}

In [5]:
count=0
for item in items:
    driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
    searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
    searchbox.send_keys(item)
    searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
    driver.execute_script("arguments[0].click();", searchbutton)
    time.sleep(2)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=89.0.4389.90)


In [9]:
driver = webdriver.Chrome("chromedriver")
driver.maximize_window() # 창 크기 최대화

item = "강남역 스시메이진"
driver.get("https://map.kakao.com/") # 카카오 지도 접속하기
searchbox = driver.find_element_by_xpath("//input[@id='search.keyword.query']") # 검색창에 입력하기
searchbox.send_keys(item)
searchbutton = driver.find_element_by_xpath("//button[@id='search.keyword.submit']") # 검색버튼 누르기
driver.execute_script("arguments[0].click();", searchbutton)
time.sleep(2)
driver.execute_script('window.open("about:blank", "_blank");') # 새 탭 열기
reviewbutton = driver.find_element_by_xpath("//a[@class='numberofscore']")
content_url = reviewbutton.get_attribute("href") 
tabs = driver.window_handles
driver.switch_to_window(tabs[1]) # 새 탭으로 이동
driver.get(content_url) # 링크 열기
"""
1페이지 리뷰 크롤링 코드 부분
"""
# 2 ~ 페이지 리뷰 이동
try:
    for i in range(2,500):
        time.sleep(2)
        another_review = driver.find_element_by_xpath("//a[@data-page='" + str(i) + "']")
        another_review.click()
        
except:
    print("더 이상 리뷰 존재 X")
driver.switch_to_window(tabs[0])
print("기본 페이지로 돌아가자")

C:\Users\zhqhd\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: use driver.switch_to.window instead
  from ipykernel import kernelapp as app


더 이상 리뷰 존재 X
기본 페이지로 돌아가자


C:\Users\zhqhd\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: use driver.switch_to.window instead
